# 실험 07-01 공통 이미터 증폭기

## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from numpy import nan, inf

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def get_simulation_result(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1
    
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1

    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def draw_plot(xs, ys, label, style_idx, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if isinstance(xs, list):
        xs = np.array(xs)
    if isinstance(ys, list):
        ys = np.array(ys)

    if (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    elif not isinstance(xs, np.ndarray):
        selected_markevery = None
        selected_marker = markers[style_idx]
    elif (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[style_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[style_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

*******************************************************************************
## 실험 1-1 결과 분석 (실험 과정 4)

### 회로도

![Markdown-mark](../experiment/circuit_01_01_01.jpeg)
![Markdown-mark](../experiment/circuit_01_01_02.jpeg)

### 1. 실험 결과에서 $I_{BQ}$, $V_{BEQ}$, $I_{CQ}$, $V_{CEQ}$를 가져와 $r_{\pi}$, $g_{m}$, $\beta_{DC}$를 계산한다.

In [ ]:
I_BQ_mA     = ?
V_VB_V      = ?
I_CQ_mA     = ?
V_VC_V      = ?

V_T_V       = 0.026

r_pi_kOhm   = V_T_V / I_BQ_mA
g_m_mA_V    = I_CQ_mA / V_T_V
beta_o      = g_m_mA_V * r_pi_kOhm

print_value('I_BQ_mA   ', I_BQ_mA)
print_value('V_VB_V    ', V_VB_V)
print_value('I_CQ_mA   ', I_CQ_mA)
print_value('V_VC_V    ', V_VC_V)

print_value('r_pi_kOhm ', r_pi_kOhm)
print_value('g_m_mA_V  ', g_m_mA_V)
print_value('beta_o    ', beta_o)

*******************************************************************************
## 실험 1-2 결과 분석 (실험 과정 6, 7)

### 회로도

![Markdown-mark](../experiment/circuit_01_02_01.jpeg)
![Markdown-mark](../experiment/circuit_01_02_02.jpeg)

### 1. 실험 결과를 가져온다.

In [ ]:
data = []

data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02_01.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02_02.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02_03.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02_04.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02_05.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02_06.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02_07.csv'))

### 2. 실험 결과에서 $V_{S}$, $V_{O}$를 가져온다.

In [ ]:
R_L_kOhm        = np.array([0.51, 1.0, 1.5, 2.0, 2.4, 3.0, 3.6])

TIME_ms_list    = []
V_S_V_list      = []
V_O_V_list      = []

for idx in range(len(R_L_kOhm)):
    TIME_ms_list.append(data[idx]['TIME'] * 1000.)
    V_S_V_list.append(data[idx]['Sample CH1'])
    V_S_V_list[-1] = savgol_filter(V_S_V_list[-1], 100, 2)
    V_O_V_list.append(data[idx]['Sample CH2'])
    V_O_V_list[-1] = savgol_filter(V_O_V_list[-1], 100, 2)

### 3. $V_{S}$와 $V_{O}$의 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_S_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.2, 2.0])
plt.ylim([-0.05, 0.05])
for idx in range(len(R_L_kOhm)):
    draw_plot(TIME_ms_list[idx], V_S_V_list[idx], 'V(VS) (R_L_kOhm=%1.3f)' % R_L_kOhm[idx], idx*2, marker_num=5)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_O_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.2, 2.0])
plt.ylim([-6.5, 6.5])
for idx in range(len(R_L_kOhm)):
    draw_plot(TIME_ms_list[idx], V_O_V_list[idx], 'V(VO) (R_L_kOhm=%1.3f)' % R_L_kOhm[idx], idx*2+1, marker_num=5)
plt.show()

### 4. $V_{S}$와 $V_{O}$의 첨두-첨두 값을 구한다.

In [ ]:
V_S_V_pp   = []
V_O_V_pp   = []

for idx in range(len(R_L_kOhm)):
    V_S_V_pp.append(V_S_V_list[idx].max() - V_S_V_list[idx].min())
    V_O_V_pp.append(V_O_V_list[idx].max() - V_O_V_list[idx].min())
V_S_V_pp = np.array(V_S_V_pp)
V_O_V_pp = np.array(V_O_V_pp)

print_array('R_L_kOhm   ', R_L_kOhm)
print_array('V_S_V_pp   ', V_S_V_pp)
print_array('V_O_V_pp   ', V_O_V_pp)

print_value('V_S_V_pp_max', V_S_V_pp.max())

### 5. 전압이득 $A_{v}$ 값을 구한다.

In [ ]:
A_v = V_O_V_pp / V_S_V_pp

print_array('R_L_kOhm   ', R_L_kOhm)
print_array('A_v        ', A_v)

### 6. $A_{v}$-$R_{L}$ 그래프를 그린다.

In [ ]:
plt.xlabel('R_L_kOhm')
plt.ylabel('A_v')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(R_L_kOhm, A_v, None, 0, marker_num=-1)

### 7. 전압이득 $A_{v}$의 시뮬레이션 결과와 계산 결과를 가져온다.

In [ ]:
A_v_sim     = np.array([     ?,      ?,      ?,      ?,      ?,      ?,      ?])
A_v_cal     = np.array([     ?,      ?,      ?,      ?,      ?,      ?,      ?])

### 8. $A_{v}$-$R_{L}$ 그래프를 그린다.

In [ ]:
plt.xlabel('R_L_kOhm')
plt.ylabel('A_v')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(R_L_kOhm, A_v, 'A_v_meas', 0, marker_num=-1)
draw_plot(R_L_kOhm, A_v_sim, 'A_v_sim', 1, marker_num=-1)
draw_plot(R_L_kOhm, A_v_cal, 'A_v_cal', 2, marker_num=-1)

*******************************************************************************
## 실험 2-1 결과 분석 (실험 과정 4)

### 회로도

![Markdown-mark](../experiment/circuit_02_01_01.jpeg)
![Markdown-mark](../experiment/circuit_02_01_02.jpeg)

### 1. 실험 결과에서 $I_{BQ}$, $V_{BEQ}$, $I_{CQ}$, $V_{CEQ}$를 가져와 $r_{\pi}$, $g_{m}$, $\beta_{DC}$를 계산한다.

In [ ]:
I_BQ_mA     = ?
V_VB_V      = ?
I_CQ_mA     = ?
V_VC_V      = ?

V_T_V       = 0.026

r_pi_kOhm   = - V_T_V / I_BQ_mA
g_m_mA_V    = - I_CQ_mA / V_T_V
beta_o      = g_m_mA_V * r_pi_kOhm

print_value('I_BQ_mA   ', I_BQ_mA)
print_value('V_VB_V    ', V_VB_V)
print_value('I_CQ_mA   ', I_CQ_mA)
print_value('V_VC_V    ', V_VC_V)

print_value('r_pi_kOhm ', r_pi_kOhm)
print_value('g_m_mA_V  ', g_m_mA_V)
print_value('beta_o    ', beta_o)

*******************************************************************************
## 실험 2-2 결과 분석 (실험 과정 6, 7)

### 회로도

![Markdown-mark](../experiment/circuit_02_02_01.jpeg)
![Markdown-mark](../experiment/circuit_02_02_02.jpeg)

### 1. 실험 결과를 가져온다.

In [ ]:
data = []

data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02_01.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02_02.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02_03.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02_04.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02_05.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02_06.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02_07.csv'))

### 2. 실험 결과에서 $V_{S}$, $V_{O}$를 가져온다.

In [ ]:
R_L_kOhm        = np.array([0.51, 1.0, 1.5, 2.0, 2.4, 3.0, 3.6])

TIME_ms_list    = []
V_S_V_list      = []
V_O_V_list      = []

for idx in range(len(R_L_kOhm)):
    TIME_ms_list.append(data[idx]['TIME'] * 1000)
    V_S_V_list.append(data[idx]['Sample CH1'])
    V_S_V_list[-1] = savgol_filter(V_S_V_list[-1], 100, 2)
    V_O_V_list.append(data[idx]['Sample CH2'])
    V_O_V_list[-1] = savgol_filter(V_O_V_list[-1], 100, 2)


### 3. $V_{S}$와 $V_{O}$의 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_S_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.2, 2.0])
plt.ylim([-0.05, 0.05])
for idx in range(len(R_L_kOhm)):
    draw_plot(TIME_ms_list[idx], V_S_V_list[idx], 'V(VS) (R_L_kOhm=%1.3f)' % R_L_kOhm[idx], idx*2, marker_num=5)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_O_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.2, 2.0])
plt.ylim([-6.5, 6.5])
for idx in range(len(R_L_kOhm)):
    draw_plot(TIME_ms_list[idx], V_O_V_list[idx], 'V(VO) (R_L_kOhm=%1.3f)' % R_L_kOhm[idx], idx*2+1, marker_num=5)
plt.show()

### 4. $V_{S}$와 $V_{O}$의 첨두-첨두 값을 구한다.

In [ ]:
V_S_V_pp   = []
V_O_V_pp   = []

for idx in range(len(R_L_kOhm)):
    V_S_V_pp.append(V_S_V_list[idx].max() - V_S_V_list[idx].min())
    V_O_V_pp.append(V_O_V_list[idx].max() - V_O_V_list[idx].min())
V_S_V_pp = np.array(V_S_V_pp)
V_O_V_pp = np.array(V_O_V_pp)

print_array('R_L_kOhm   ', R_L_kOhm)
print_array('V_S_V_pp   ', V_S_V_pp)
print_array('V_O_V_pp   ', V_O_V_pp)

print_value('V_S_V_pp_max', V_S_V_pp.max())

### 5. 전압이득 $A_{v}$ 값을 구한다.

In [ ]:
A_v = V_O_V_pp / V_S_V_pp

print_array('R_L_kOhm   ', R_L_kOhm)
print_array('A_v        ', A_v)

### 6. $A_{v}$-$R_{L}$ 그래프를 그린다.

In [ ]:
plt.xlabel('R_L_kOhm')
plt.ylabel('A_v')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(R_L_kOhm, A_v, None, 0, marker_num=-1)

### 7. 전압이득 $A_{v}$의 시뮬레이션 결과와 계산 결과를 가져온다.

In [ ]:
A_v_sim     = np.array([     ?,      ?,      ?,      ?,      ?,      ?,      ?])
A_v_cal     = np.array([     ?,      ?,      ?,      ?,      ?,      ?,      ?])

### 8. $A_{v}$-$R_{L}$ 그래프를 그린다.

In [ ]:
plt.xlabel('R_L_kOhm')
plt.ylabel('A_v')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(R_L_kOhm, A_v, 'A_v_meas', 0, marker_num=-1)
draw_plot(R_L_kOhm, A_v_sim, 'A_v_sim', 1, marker_num=-1)
draw_plot(R_L_kOhm, A_v_cal, 'A_v_cal', 2, marker_num=-1)